# Generating Heatmap to Show Dengue Sufferers On Map of India 

### Generate the data

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Create a function to generate a dataset
def generate_dengue_data(num_entries, start_date):
    # Generate random data
    dates = [start_date + timedelta(days=i) for i in range(num_entries)]
    regions = ['North', 'South', 'East', 'West', 'Central']
    cases = np.random.randint(10, 100, size=num_entries)
    deaths = np.random.randint(0, 10, size=num_entries)
    region = [np.random.choice(regions) for _ in range(num_entries)]
    
    # Create DataFrame
    data = {
        'Date': dates,
        'Region': region,
        'Cases': cases,
        'Deaths': deaths
    }
    df = pd.DataFrame(data)
    return df

# Generate a dataset with 100 entries starting from January 1, 2024
num_entries = 100
start_date = datetime(2024, 1, 1)
dengue_data = generate_dengue_data(num_entries, start_date)

# Save to CSV
csv_file_path = 'dengue_data.csv'
dengue_data.to_csv(csv_file_path, index=False)



csv_file_path


'dengue_data.csv'

### Generate the heatmap

In [3]:
import folium
import pandas as pd
import geopandas as gpd
from folium.plugins import HeatMap
from IPython.display import display, IFrame

# Load the dengue data
df = pd.read_csv('dengue_data.csv')

# Load India's states geojson data
# Assuming 'india_states.geojson' is available in the same directory
india_geojson = 'india_state_geo.json'  # Replace with the actual path to the GeoJSON file
india_map = gpd.read_file(india_geojson)

# Simplify the dataset for the heatmap
# Aggregate data to sum cases and deaths per region
heatmap_data = df.groupby('Region').agg({'Cases': 'sum', 'Deaths': 'sum'}).reset_index()

# Add latitude and longitude for the regions
# This example uses hypothetical coordinates; you need to replace these with actual ones
region_coords = {
    'North': [28.6139, 77.2090],  # Delhi
    'South': [12.9716, 77.5946],  # Bangalore
    'East': [22.5726, 88.3639],   # Kolkata
    'West': [19.0760, 72.8777],   # Mumbai
    'Central': [23.2599, 77.4126] # Bhopal
}

# Merge coordinates with the heatmap data
heatmap_data['Latitude'] = heatmap_data['Region'].apply(lambda x: region_coords[x][0])
heatmap_data['Longitude'] = heatmap_data['Region'].apply(lambda x: region_coords[x][1])

# Initialize a map centered around India
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

# Add heatmap
heat_data = [[row['Latitude'], row['Longitude'], row['Cases']] for index, row in heatmap_data.iterrows()]
HeatMap(heat_data, radius=15, blur=10, max_zoom=1).add_to(m)

# Save the map as an HTML file
map_file = 'dengue_heatmap_india.html'
m.save(map_file)

# Display the map in the notebook
display(IFrame(map_file, width=600, height=600))
